### Columns to drop
- building_metadata - drop floor count, year built
- weather_train - precipitation depth 1_hr, cloud coverage

### Sites to Drop
- see files for electricity, steam, gas

### SQL Query

In [15]:
import psycopg2
from psycopg2 import Error

import pandas as pd
import numpy as np

import os
path = os.path.join(os.path.expanduser('~'), 'Documents', 'github.passwords', 'energy.predictor.2020.config')
print (path)

myfile = open(path,"rt")
config= myfile.read()

/home/douglas/Documents/github.passwords/energy.predictor.2020.config


In [16]:
connection = psycopg2.connect(config)

cursor = connection.cursor()

In [17]:
# Read PostgreSQL into a dataframe
sql =''' SELECT 
            bm.site_id,
            bm.building_id,
            bm.primary_use,
            t.meter,
            t.meter_reading,
            w.timestamp,
            w.air_temperature,
            w.dew_temperature,
            w.sea_level_pressure,
            w.wind_direction,
            w.wind_speed
         FROM 
             weather_train AS w
         INNER JOIN building_metadata AS bm
             ON (bm.site_id = w.site_id)
         INNER JOIN train AS t
             ON (t.timestamp = w.timestamp) AND (t.building_id = bm.building_id)
         WHERE 
             w.timestamp IS NOT NULL 
             AND w.air_temperature IS NOT NULL 
             AND w.dew_temperature IS NOT NULL
             AND w.sea_level_pressure IS NOT NULL 
             AND w.wind_direction IS NOT NULL 
             AND w.wind_speed IS NOT NULL
            ;'''
df = pd.read_sql_query(sql,connection)

### Drop any observation that has a null

In [18]:
if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


### Preprocessing Here

Wind direction preprocessing

Time preprocessing

In [19]:
df

,site_id,building_id,primary_use,meter,meter_reading,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,0,Education,0,0.0000,2016-01-01,25.0,20.0,1019.7,0.0,0.0
1,0,1,Education,0,0.0000,2016-01-01,25.0,20.0,1019.7,0.0,0.0
2,0,2,Education,0,0.0000,2016-01-01,25.0,20.0,1019.7,0.0,0.0
3,0,3,Education,0,0.0000,2016-01-01,25.0,20.0,1019.7,0.0,0.0
4,1,147,Office,0,72.0612,2016-01-01,3.8,2.4,1020.9,240.0,3.1
...,...,...,...,...,...,...,...,...,...,...,...
17684649,1,144,Office,0,28.0638,2016-01-01,3.8,2.4,1020.9,240.0,3.1
17684650,1,144,Office,3,9.7989,2016-01-01,3.8,2.4,1020.9,240.0,3.1
17684651,1,145,Office,0,9.4350,2016-01-01,3.8,2.4,1020.9,240.0,3.1
17684652,1,145,Office,3,100.0000,2016-01-01,3.8,2.4,1020.9,240.0,3.1
